In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut
import copy



In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in M

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Gefitinib'
na = -1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [5]:
label_r=label_r.fillna(na)

In [6]:
hvg,adata = ut.highly_variable_genes(data_r)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [7]:
label_r

,Erlotinib,AICAR,Camptothecin,Vinblastine,Cisplatin,Cytarabine,Docetaxel,Methotrexate,ATRA,Gefitinib,...,CMK,Pyrimethamine,JW-7-52-1,A-443654,GW843682X,MS-275,Parthenolide,MG-132,KIN001-135,TGX221
BxPC-3,-1.000000,0.159708,0.188393,0.436127,0.154274,0.113764,0.249987,0.015324,0.015324,0.020293,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
KMOE-2,0.002108,0.339814,0.318206,0.281740,0.063808,0.140681,0.101530,0.191210,0.031229,0.007419,...,0.022662,0.025217,0.270791,0.095608,0.139914,0.171747,0.026670,0.013904,0.002116,0.000956
MFM-223,0.002148,0.334087,0.122671,0.361438,0.051983,0.186659,0.228797,0.003639,0.023043,0.003639,...,0.038997,0.006167,0.002185,0.102592,0.020338,0.010874,0.000970,0.035206,0.007051,0.000323
NUGC-3,-1.000000,0.095099,0.416792,0.422631,0.266409,0.237811,0.234952,0.011558,0.028838,0.153002,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
OC-314,-1.000000,0.012740,0.479915,0.390145,0.067738,0.125869,0.249668,0.110418,0.002214,0.020309,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KP-N-S19s,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
BC-3,0.003515,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,0.048227,0.146476,0.400087,0.205247,0.147090,0.298803,0.023796,0.017219,0.002065,0.069181
Panc 08.13,-1.000000,0.152818,0.196279,0.180653,0.090963,0.093197,0.097267,0.018269,0.010237,0.009405,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
EKVX,0.177238,0.078395,0.078938,0.072213,0.007225,0.003470,0.129184,0.095642,0.111598,0.126364,...,0.020472,0.044047,0.243635,0.079375,0.019859,0.177759,0.014843,0.114431,0.007216,0.063911


In [8]:
selected_idx = label_r.loc[:,select_drug]!=na

In [9]:
data_r.columns = adata.var_names

# Your is gene-cell, mine is cell-gene

In [10]:
#data = data_r.loc[selected_idx,:]
data = data_r.loc[selected_idx,hvg]

In [11]:
label = label_r.loc[selected_idx,select_drug]
#sscaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
mmscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
#data = sscaler.fit_transform(data)


In [12]:
print(np.std(data))
print(np.mean(data))

0.25383929199289434
0.2727100504700379


In [13]:
data.mean(axis=0)


array([0.49458949, 0.09860014, 0.21159545, ..., 0.21276445, 0.22738253,
       0.27747885])

In [14]:
print(data.max())
print(data.min())

1.0000000000000004
0.0


In [15]:
data.shape

(675, 3462)

In [16]:
label_r.shape

(789, 139)

# Split test train

In [17]:
from sklearn.model_selection import train_test_split
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.2, random_state=42)

In [18]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(675, 3462)
(675,)
(432, 3462) (432,)
(135, 3462) (135,)


In [19]:
print(X_train.max())
print(X_train.min())

1.0000000000000002
0.0


# AE MODEL

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [21]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)
#X_alltrainTensor = torch.FloatTensor(X_train_all).to(device)


Y_trainTensor = torch.FloatTensor(Y_train.values).to(device)
Y_validTensor = torch.FloatTensor(Y_valid.values).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [22]:
dataloader = X_trainDataLoader

In [23]:
dataloaders_train = {'train':X_trainDataLoader,'val':X_validDataLoader}

In [24]:
X_trainDataLoader.dataset.tensors[0].shape[0]

432

In [25]:
{x: dataloaders_train[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}

{'train': 432, 'val': 108}

# The model

In [26]:
model = AEBase(input_dim=data.shape[1],latent_dim=512,hidden_dims=[2048,1024])

In [27]:
print(model)

AEBase(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3462, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=2048, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=1024, out_features=512, bias=True)
  (decoder_input): Linear(in_features=512, out_features=1024, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1024, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (final_layer): Sequential(
    (0): Linear(in_features=2048, out_features=3462, bias=True)
    (1): Sigmoid(

In [28]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [29]:
def train_ae_model(net,data_loaders,optimizer,loss_function,n_epochs,scheduler):
    
    dataset_sizes = {x: data_loaders[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}
    loss_train = {}
    
    best_model_wts = copy.deepcopy(net.state_dict())
    best_loss = np.inf

    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = scheduler(optimizer, epoch)
                net.train()  # Set model to training mode
            else:
                net.eval()  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            # for data in data_loaders[phase]:
            for batchidx, (x, _) in enumerate(data_loaders[phase]):

                x.requires_grad_(True)
                # encode and decode 
                output = model(x)
                # compute loss
                loss = loss_function(output, x)      

                # zero the parameter (weight) gradients
                optimizer.zero_grad()

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    # update the weights
                    optimizer.step()

                # print loss statistics
                running_loss += loss.item()
            
            # Schedular
#             if phase == 'train':
#                 scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)
                
            last_lr = scheduler.optimizer.param_groups[0]['lr']
            loss_train[epoch,phase] = epoch_loss
            print('{} Loss: {:.8f}. Learning rate = {}'.format(phase, epoch_loss,last_lr))
            
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
    
    # Select best model wts
    torch.save(best_model_wts, 'saved/models/ae.pkl')
    net.load_state_dict(best_model_wts)           
    
    return net, loss_train

In [30]:
model,loss_report = train_ae_model(net=model,data_loaders=dataloaders_train,
                             optimizer=optimizer,loss_function=loss_function,
                            n_epochs=epochs,scheduler=exp_lr_scheduler)

Epoch 0/499
----------
train Loss: 0.00137978. Learning rate = 0.01
val Loss: 0.00364676. Learning rate = 0.01
Epoch 1/499
----------
train Loss: 0.00179373. Learning rate = 0.01
val Loss: 0.00209717. Learning rate = 0.01
Epoch 2/499
----------
train Loss: 0.00126729. Learning rate = 0.01
val Loss: 0.00199299. Learning rate = 0.01
Epoch 3/499
----------
train Loss: 0.00118850. Learning rate = 0.01
val Loss: 0.00184282. Learning rate = 0.01
Epoch 4/499
----------
train Loss: 0.00110372. Learning rate = 0.01
val Loss: 0.00183227. Learning rate = 0.01
Epoch 5/499
----------
train Loss: 0.00103471. Learning rate = 0.01
val Loss: 0.00167147. Learning rate = 0.01
Epoch 6/499
----------
train Loss: 0.00099253. Learning rate = 0.01
val Loss: 0.00165997. Learning rate = 0.01
Epoch 7/499
----------
train Loss: 0.00098848. Learning rate = 0.01
val Loss: 0.00158414. Learning rate = 0.01
Epoch 8/499
----------
train Loss: 0.00094238. Learning rate = 0.01
val Loss: 0.00139948. Learning rate = 0.01
E

train Loss: 0.00024096. Learning rate = 0.01
val Loss: 0.00033422. Learning rate = 0.01
Epoch 76/499
----------
train Loss: 0.00026034. Learning rate = 0.01
val Loss: 0.00032989. Learning rate = 0.01
Epoch 77/499
----------
train Loss: 0.00025672. Learning rate = 0.01
val Loss: 0.00031021. Learning rate = 0.01
Epoch 78/499
----------
train Loss: 0.00024812. Learning rate = 0.01
val Loss: 0.00030526. Learning rate = 0.01
Epoch 79/499
----------
train Loss: 0.00023394. Learning rate = 0.01
val Loss: 0.00029488. Learning rate = 0.01
Epoch 80/499
----------
train Loss: 0.00023871. Learning rate = 0.01
val Loss: 0.00029169. Learning rate = 0.01
Epoch 81/499
----------
train Loss: 0.00023021. Learning rate = 0.01
val Loss: 0.00029095. Learning rate = 0.01
Epoch 82/499
----------
train Loss: 0.00022611. Learning rate = 0.01
val Loss: 0.00028766. Learning rate = 0.01
Epoch 83/499
----------
train Loss: 0.00022493. Learning rate = 0.01
val Loss: 0.00028437. Learning rate = 0.01
Epoch 84/499
---

val Loss: 0.00022599. Learning rate = 0.01
Epoch 151/499
----------
train Loss: 0.00017774. Learning rate = 0.01
val Loss: 0.00022601. Learning rate = 0.01
Epoch 152/499
----------
train Loss: 0.00017832. Learning rate = 0.01
val Loss: 0.00022525. Learning rate = 0.01
Epoch 153/499
----------
train Loss: 0.00017270. Learning rate = 0.01
val Loss: 0.00022418. Learning rate = 0.01
Epoch 154/499
----------
train Loss: 0.00017186. Learning rate = 0.01
val Loss: 0.00022401. Learning rate = 0.01
Epoch 155/499
----------
train Loss: 0.00017544. Learning rate = 0.01
val Loss: 0.00022398. Learning rate = 0.01
Epoch 156/499
----------
train Loss: 0.00016777. Learning rate = 0.01
val Loss: 0.00022298. Learning rate = 0.01
Epoch 157/499
----------
train Loss: 0.00017066. Learning rate = 0.01
val Loss: 0.00022374. Learning rate = 0.01
Epoch 158/499
----------
train Loss: 0.00017189. Learning rate = 0.01
val Loss: 0.00022140. Learning rate = 0.01
Epoch 159/499
----------
train Loss: 0.00016701. Lear

train Loss: 0.00014764. Learning rate = 0.001
val Loss: 0.00020362. Learning rate = 0.001
Epoch 226/499
----------
train Loss: 0.00014856. Learning rate = 0.001
val Loss: 0.00020358. Learning rate = 0.001
Epoch 227/499
----------
train Loss: 0.00014790. Learning rate = 0.001
val Loss: 0.00020353. Learning rate = 0.001
Epoch 228/499
----------
train Loss: 0.00014898. Learning rate = 0.001
val Loss: 0.00020345. Learning rate = 0.001
Epoch 229/499
----------
train Loss: 0.00015020. Learning rate = 0.001
val Loss: 0.00020349. Learning rate = 0.001
Epoch 230/499
----------
train Loss: 0.00014813. Learning rate = 0.001
val Loss: 0.00020347. Learning rate = 0.001
Epoch 231/499
----------
train Loss: 0.00014647. Learning rate = 0.001
val Loss: 0.00020355. Learning rate = 0.001
Epoch 232/499
----------
train Loss: 0.00014510. Learning rate = 0.001
val Loss: 0.00020349. Learning rate = 0.001
Epoch 233/499
----------
train Loss: 0.00014751. Learning rate = 0.001
val Loss: 0.00020345. Learning rat

train Loss: 0.00014658. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020328. Learning rate = 1.0000000000000004e-08
Epoch 292/499
----------
train Loss: 0.00014628. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020334. Learning rate = 1.0000000000000004e-08
Epoch 293/499
----------
train Loss: 0.00015223. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020331. Learning rate = 1.0000000000000004e-08
Epoch 294/499
----------
train Loss: 0.00015105. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020329. Learning rate = 1.0000000000000004e-08
Epoch 295/499
----------
train Loss: 0.00014752. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020323. Learning rate = 1.0000000000000004e-08
Epoch 296/499
----------
train Loss: 0.00015015. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020331. Learning rate = 1.0000000000000004e-08
Epoch 297/499
----------
train Loss: 0.00015074. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020342. Learning rate = 1.000

train Loss: 0.00014893. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020354. Learning rate = 1.0000000000000004e-08
Epoch 349/499
----------
train Loss: 0.00014790. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020345. Learning rate = 1.0000000000000004e-08
Epoch 350/499
----------
train Loss: 0.00014891. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020344. Learning rate = 1.0000000000000004e-08
Epoch 351/499
----------
train Loss: 0.00014791. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020334. Learning rate = 1.0000000000000004e-08
Epoch 352/499
----------
train Loss: 0.00014814. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020334. Learning rate = 1.0000000000000004e-08
Epoch 353/499
----------
train Loss: 0.00014623. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020335. Learning rate = 1.0000000000000004e-08
Epoch 354/499
----------
train Loss: 0.00014713. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020331. Learning rate = 1.000

train Loss: 0.00015457. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020326. Learning rate = 1.0000000000000004e-08
Epoch 406/499
----------
train Loss: 0.00014818. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020332. Learning rate = 1.0000000000000004e-08
Epoch 407/499
----------
train Loss: 0.00015086. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020331. Learning rate = 1.0000000000000004e-08
Epoch 408/499
----------
train Loss: 0.00015051. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020326. Learning rate = 1.0000000000000004e-08
Epoch 409/499
----------
train Loss: 0.00014964. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020330. Learning rate = 1.0000000000000004e-08
Epoch 410/499
----------
train Loss: 0.00014922. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020335. Learning rate = 1.0000000000000004e-08
Epoch 411/499
----------
train Loss: 0.00014827. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020341. Learning rate = 1.000

train Loss: 0.00014830. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020325. Learning rate = 1.0000000000000004e-08
Epoch 463/499
----------
train Loss: 0.00014783. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020328. Learning rate = 1.0000000000000004e-08
Epoch 464/499
----------
train Loss: 0.00014716. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020325. Learning rate = 1.0000000000000004e-08
Epoch 465/499
----------
train Loss: 0.00014953. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020329. Learning rate = 1.0000000000000004e-08
Epoch 466/499
----------
train Loss: 0.00015013. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020341. Learning rate = 1.0000000000000004e-08
Epoch 467/499
----------
train Loss: 0.00014932. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020341. Learning rate = 1.0000000000000004e-08
Epoch 468/499
----------
train Loss: 0.00015026. Learning rate = 1.0000000000000004e-08
val Loss: 0.00020343. Learning rate = 1.000

In [31]:
loss_report

{(0, 'train'): 0.0013797760561660485,
 (0, 'val'): 0.0036467586954434714,
 (1, 'train'): 0.001793733142592289,
 (1, 'val'): 0.002097165005074607,
 (2, 'train'): 0.0012672857070962589,
 (2, 'val'): 0.001992992642852995,
 (3, 'train'): 0.0011884989561858,
 (3, 'val'): 0.001842819016288828,
 (4, 'train'): 0.001103721455567413,
 (4, 'val'): 0.0018322684422687249,
 (5, 'train'): 0.001034714109091847,
 (5, 'val'): 0.0016714729092739246,
 (6, 'train'): 0.0009925312328117866,
 (6, 'val'): 0.0016599682470162709,
 (7, 'train'): 0.0009884778065262017,
 (7, 'val'): 0.0015841446540973804,
 (8, 'train'): 0.000942378960273884,
 (8, 'val'): 0.001399482289950053,
 (9, 'train'): 0.0009845559899177817,
 (9, 'val'): 0.0013086842717947784,
 (10, 'train'): 0.0009073166659584751,
 (10, 'val'): 0.0012631921304596795,
 (11, 'train'): 0.0009083885147615716,
 (11, 'val'): 0.0011532302531931135,
 (12, 'train'): 0.000908859315569754,
 (12, 'val'): 0.0010903419719802008,
 (13, 'train'): 0.0008567588541794706,
 (13,

In [32]:
# # Old code for autoencoders
# for epoch in range(epochs):
#     # 不需要label，所以用一个占位符"_"代替
#     for batchidx, (x, _) in enumerate(X_allDataLoader):
#         x.requires_grad_(True)
#         # encode and decode 
#         output = model(x)
#         # compute loss
#         print(output.shape)
#         loss = loss_function(output, x)      
#         # update
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
           
#     loss_train[epoch,0] = loss.item()  
#     print('Epoch: %04d, Training loss=%.8f' %
#           (epoch+1, loss.item()))

In [ ]:
# torch.save(model.state_dict(), 'saved/models/ae.pkl')

In [33]:
recon_batch = model(X_trainTensor)

In [34]:
loss_function(recon_batch,X_trainTensor)

tensor(0.0208, device='cuda:0', grad_fn=<MseLossBackward>)

In [35]:
recon_batch

tensor([[0.5086, 0.0377, 0.1248,  ..., 0.2024, 0.1449, 0.4762],
        [0.5402, 0.0980, 0.1292,  ..., 0.3138, 0.2818, 0.2321],
        [0.0515, 0.4554, 0.0792,  ..., 0.1940, 0.4289, 0.1500],
        ...,
        [0.4740, 0.0804, 0.3108,  ..., 0.1630, 0.2161, 0.1823],
        [0.1886, 0.1683, 0.0924,  ..., 0.2058, 0.2563, 0.3026],
        [0.6708, 0.0478, 0.1876,  ..., 0.2436, 0.1315, 0.4909]],
       device='cuda:0', grad_fn=<SigmoidBackward>)

In [36]:
X_trainTensor

tensor([[0.7359, 0.0868, 0.4233,  ..., 0.2414, 0.1303, 0.6761],
        [0.5435, 0.0496, 0.0739,  ..., 0.1036, 0.2022, 0.3589],
        [0.1336, 0.4184, 0.0618,  ..., 0.1218, 0.3170, 0.3572],
        ...,
        [0.3869, 0.0693, 0.4479,  ..., 0.2760, 0.2085, 0.0863],
        [0.1415, 0.0232, 0.0663,  ..., 0.2076, 0.3566, 0.3610],
        [0.6655, 0.0682, 0.3321,  ..., 0.2687, 0.1723, 0.2549]],
       device='cuda:0')

In [ ]:
recon_batch.cpu().detach().numpy().min()

In [ ]:
train_embeddings = model.encode(X_trainTensor)

In [ ]:
feature = train_embeddings.cpu().detach().numpy()

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.001)
clf.fit(feature, Y_train.values)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFrg = RandomForestRegressor(random_state=0, n_estimators=100)
RFrg.fit(feature, Y_train.values)

In [ ]:
testFeature = model.encode(X_testTensor)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [ ]:
rfresult = RFrg.predict(testFeature.detach().cpu().numpy())

In [ ]:
r2_score(lasso,Y_test)

In [ ]:
r2_score(rfresult,Y_test)

In [ ]:
Y_test.values

In [ ]:
rfresult

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
tsne_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_ae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [ ]:
# Initialization DNN model

predictor = DNN(128, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss()
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [ ]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

In [ ]:
testpredict = predictor(testFeature)

In [ ]:
testpredict

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)